In [18]:
import numpy as np
import pandas as pd
import math
import os
import swifter
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
from names_dataset import NameDataset
import functools
import operator
from joblib import Parallel, delayed


In [2]:
os.chdir("C:\\Users\\fjying\\Desktop\\RA_Columbia\\patentparsing\\input_2012")

In [3]:
#import the dataset
patents = pd.read_csv("ad20121231-03.csv")

C:\Users\fjying\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,22,23,24,25,26,27,28,29,30,31,33,34,35,36,37,38,39,40,41,42,43,44,45,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,75,76,77,78,79,80,81,82,87,88,89,90,91,92,93,94,95,96,98,99,100,101,102,103,104,105,106,107,108,109,110,113,124,129,130,131,132,133,134,135,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
#preprocess company name
#Convert company names to standardized name for better matching

In [6]:
def name_preprocess(x):
    if type(x) != str:
        return x
    x = x.lower().replace(",", "").replace("-", "").replace("*", "").replace("!", "").replace("(", "").replace(")", "")
    x = x.replace("'", "").replace(";", "").replace("/", "").replace("-", "").replace(r"\(.*\)", "").replace("u.s", "us")
    x = x.replace("u s", "us").replace(".", "").strip()
    return x

In [7]:
#drop endwords in the company name; remove influence of these words on matching
word_lists = ["inc", "company", "companies", "corporation", "corporations", "com", "co", "ltd", "co ltd", "corp", "gmbh", "group", 
              "gp", "plc",  "cl a", "cl b", "cl i", "-redh", "/de", "/md", "&", "lp", "llc", "limited liability",
              "svc", "holdings", "international", "incorporated", 'financial', "technology", "technologies", 
              "limited", "cdti", "sa", "bv", "sl", 
              "a/s", "the", "gmbh", "nv", "ag", "kg", "holding"]

#drop end words which do not have spaces between words
def dropendwords(x, word_lists):
    if type(x) != str:
        return x
    for word in word_lists:
        length = len(word)
        if x[-length:] == word:
            return x[0:-length].strip()
    return x


In [4]:
#sort patents by patent ids and last update date
patents = patents.sort_values(by =['patent_ids', 'last_update_date'])

In [5]:
#drop duplicated entry
patents = patents.drop_duplicates( ['patent_ids', 'assignor', 'name_0'], keep = 'first')

In [ ]:
patentids = np.unique(patents['patent_ids'].values)

In [8]:
names = patents.head(1).sort_index(axis = 1).columns

In [9]:
names = [s for s in names if s[0:4] =='name']
names.append('assignor')

In [10]:
for name in names:
    patents[name] = patents[name].swifter.apply(lambda x: name_preprocess(x)).swifter.apply(lambda x: dropendwords(x, word_lists)).swifter.apply(lambda x: dropendwords(x, word_lists))

In [ ]:
#identify patent transfer by fuzzy match assignor names/address with assignee names/address

In [11]:
def identify_notmeaningful_one(temp):
        match = False
        for i in range(temp['total_number_assignees']):
            if (fuzz.token_sort_ratio(temp['assignor'],temp['name_' + str(i)]) >= 80):
                match = True
                break
            if set(temp['assignor'].split()) & set(temp['name_' + str(i)].split()):
                match = True
                break
        return match

In [12]:
#Process each patent id
def identify_patent_not_meaningful(temp):
    notmeaningful = []
    addresses = ['country-name','state', 'city', 'postcode','address-2', 'address-1']
    #score threshold is the simialirty threshold
    for i, assignment in temp.iterrows():
        assignor  = assignment['assignor']
        try:
            idx = list(temp['name_0']).index(assignor)
            assignor_ad = temp.iloc[idx]
            scores = []
            for i in range(assignment['total_number_assignees']):
                for address in addresses:
                    if address + '_0' in assignor_ad.keys() and address + '_' + str(i) in assignment.keys():
                        score = fuzz.token_sort_ratio(assignor_ad[address + '_0' ], assignment[address + '_' + str(i)])
                        if score < 85:
                            break       
                if score >= 85:
                    notmeaningful.append(True)
                    break
            if score < 85:
                notmeaningful.append(False)
        except ValueError:
            match = identify_notmeaningful_one(assignment)
            notmeaningful.append(match)
    return notmeaningful
            

In [35]:
def final_identify_not_meaningful(patentid): 
    temp = patents[patents['patent_ids'] == patentid]
    #drop empty columns
    temp = temp.dropna(how = 'all', axis = 1)
    notmeaningful = []
    if temp.shape[0] == 1:
        #compare name directly
        match0 = identify_notmeaningful_one(temp.iloc[0])    
        notmeaningful.append(match0)
    else:
        match1 = identify_patent_not_meaningful(temp)
        notmeaningful.extend(match1)
    return notmeaningful

In [ ]:
with Parallel(n_jobs=-1, verbose=1, prefer="threads") as parallel:
    results = parallel(delayed(final_identify_not_meaningful)(patentid) for patentid in patentids)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   23.1s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   50.0s
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 2442 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done 3192 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 4042 tasks      | elapsed:  7.3min
[Parallel(n_jobs=-1)]: Done 4992 tasks      | elapsed:  8.7min
[Parallel(n_jobs=-1)]: Done 6042 tasks      | elapsed: 10.3min
[Parallel(n_jobs=-1)]: Done 7192 tasks      | elapsed: 12.1min
[Parallel(n_jobs=-1)]: Done 8442 tasks      | elapsed: 14.0min
[Parallel(n_jobs=-1)]: Done 9792 tasks      | elapsed: 16.4min
[Parallel(n_jobs=-1)]: Done 11242 tasks  

In [37]:
notmeaningful = functools.reduce(operator.iconcat, results, [])
patents['notmeaningful'] = notmeaningful

[True,
 True,
 False,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 False,
 True,
 True,
 True,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 False,
 True,
 False,
 True,
 False,
 False,
 True,
 False,
 True]

In [ ]:
columns1.extend(set(names)-set(columns1))
columns1

In [ ]:
patents = patents[columns1]

In [ ]:
patents['assignor'] = patents['assignor'].astype(str)
patents['name_0'] = patents['name_0'].astype(str)

In [ ]:
#reorganize columsn in patent data
names = patents.head(1).sort_index(axis = 1).columns
columns1 = ['notmeaningful', 'patent_ids','assignor', 'last_update_date', 'reel-no', 'frame-no', 'total_number_assignees',
            'name_0','address-1_0', 'address-2_0', 'city_0','country-name_0','postcode_0','state_0']

In [ ]:
#Create the unique id for each patent
#Multiple patent assignemnts of one patent may be updated at the same date
patents['groupcount'] = patents.groupby(['patent_ids', 'last_update_date']).cumcount()
patents['uniqueid'] = patents['groupcount'].astype(str)
patents = patents.drop(['groupcount'], axis = 1)

In [ ]:
patents_sort= patents[['patent_ids','assignor', 'name_0', 'last_update_date', 'uniqueid']].sort_values(['patent_ids', 'last_update_date'])

In [ ]:
patents_sort_find = patents_sort.drop_duplicates(['patent_ids'], keep = 'first')

In [ ]:
simplest = patents_sort_find.drop_duplicates(['assignor', 'name_0'])

In [ ]:
#Import the dataset of names
m = NameDataset()

In [ ]:
def identify_name(x):
    if m.search_last_name(x) or m.search_first_name(x):
        return True
    else:
        return False

In [ ]:
def identify_person(df):
    temp = df['assignor'].split(' ')
    if len(temp[-1]) == 1 and temp[-1].isalpha() and len(temp) == 3 and sum([identify_name(name) for name in temp[:-1]]) == 2:
        return True
    elif sum([identify_name(name) for name in temp]) == len(temp) and len(temp)>=2:
        return True
    return False
    

In [ ]:
simplest['notmeanigful_firsttransfer'] = simplest.apply(identify_person,axis = 1)

In [ ]:
simplest['assignor_name_0'] = simplest['assignor']+ simplest['name_0']
simplest= simplest.drop(['assignor', 'name_0'], axis = 1)

In [ ]:
patents_sort_find['assignor_name_0'] = patents_sort_find['assignor']+ patents_sort_find['name_0']

In [ ]:
patents_sort_find = pd.merge(patents_sort_find,simplest[['notmeanigful_firsttransfer', 'assignor_name_0', 'uniqueid']], how = 'left', on = 'assignor_name_0')
patents_sort_find = patents_sort_find.drop(['assignor_name_0', 'uniqueid_y'], axis = 1)
patents_sort_find = patents_sort_find.rename(columns = {'uniqueid_x': 'uniqueid'})

In [ ]:
patents_2 = pd.merge(patents, patents_sort_find[['notmeanigful_firsttransfer', 'uniqueid']], on = 'uniqueid', how = 'left')

In [ ]:
patents_2['notmeanigful_firsttransfer'] = patents_2['notmeanigful_firsttransfer'].fillna(False)

In [ ]:
patents_2.loc[patents_2['notmeanigful_firsttransfer'], 'notmeaningful'] = True

In [ ]:
patents_2[patents_2['notmeanigful_firsttransfer']][['assignor', 'name_0', 'notmeaningful', 'notmeanigful_firsttransfer']]

In [ ]:
#reconstruct column names


In [ ]:
patents_2.to_csv("2015_notmeaningfultransfer.csv")